<a href="https://colab.research.google.com/github/Mohammad-Rahmdel/SIGNS_Recognition_Using_Tensorflow/blob/master/SIGNS_Recognition_with_Regularization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyDrive

     |████████████████████████████████| 993kB 7.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1UOzbDqDkxNY0Sa9tKxalxp8kWIHxSOKL'})
download.GetContentFile('tf_utils.py')


In [0]:
!mkdir datasets 

download = drive.CreateFile({'id': '1fCmaeQyBWf9RFsaGdRYiAG-BypTzN8Nu'})
download.GetContentFile('datasets/test_signs.h5')

download = drive.CreateFile({'id': '1nU3KQ7fdTcfHNgY8jk9xbhqYlxDcWiiu'})
download.GetContentFile('datasets/train_signs.h5')


In [13]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # ignoring warnings

# Problem statement: SIGNS Dataset 

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
# Flattening
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T
# Normalizing
X_train = X_train_flatten/255.
X_test = X_test_flatten/255.
# Converting labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6)
Y_test = convert_to_one_hot(Y_test_orig, 6)


"""
Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.

Input(12288) -> Relu(number of hiddent units = 25) -> Relu(number of hiddent units = 12) -> 
Relu(number of output units = c = 6) 
b1 = (25, 1)
W1 = (25, 12288)
b2 = (12, 1)
W1 = (12, 25)
b3 = (6, 1)
W3 = (6, 12)



Arguments:
X_train (input size = 12288, number of training examples = 1080)
Y_train (output size = 6, number of training examples = 1080)
X_test (12288, 120)
Y_test (6, 120)
"""

learning_rate = 0.0001
num_epochs = 1000
minibatch_size = 32
print_cost = True
keep_prob = 1
lambd = 0
    
ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
tf.set_random_seed(1)                             # to keep consistent results
seed = 3                    

m = Y_train.shape[1]
n_x = X_train.shape[0]
n_y = Y_train.shape[0]



X = tf.placeholder(shape=[n_x, None] ,dtype=tf.float32, name='X')
Y = tf.placeholder(shape=[n_y, None] ,dtype=tf.float32, name='Y')

tf.set_random_seed(1)                  
b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
b3 = tf.get_variable("b3", [6,1], initializer = tf.zeros_initializer())

W1 = tf.get_variable("W1", [25,12288], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
W2 = tf.get_variable("W2", [12,25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
W3 = tf.get_variable("W3", [6,12], initializer = tf.contrib.layers.xavier_initializer(seed = 1))


Z1 = tf.add(tf.matmul(W1, X) , b1)
A1 = tf.nn.relu(Z1)
A1 = tf.nn.dropout(A1, keep_prob)
Z2 = tf.add(tf.matmul(W2, A1) , b2)
A2 = tf.nn.relu(Z2)
A2 = tf.nn.dropout(A2, keep_prob)
Z3 = tf.add(tf.matmul(W3, A2) , b3)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = tf.transpose(Z3), labels = tf.transpose(Y) ) + 
    lambd * tf.nn.l2_loss(W1) +
    lambd * tf.nn.l2_loss(W2) +
    lambd * tf.nn.l2_loss(W3) )


costs = [] 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.global_variables_initializer() # Initialize all the variables

with tf.Session() as session:
    session.run(init)
    for i in range(num_epochs):
        # seed = seed + 1
        epoch_cost = 0.
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
        n_minibatches = np.floor(m / minibatch_size) # 1080 / 32 = 33.75
        for minibatch in minibatches:
            (minibatch_X, minibatch_Y) = minibatch
            _ , minibatch_cost = session.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
            epoch_cost += minibatch_cost / n_minibatches

        
        if print_cost == True:
            if i % 100 == 0:
                print ("Cost after epoch %i: %f" % (i, epoch_cost))
            if i % 5 == 0:
                costs.append(epoch_cost)
        
    
  
    correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
    print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))

Cost after epoch 0: 1.867018
Cost after epoch 100: 1.030552
Cost after epoch 200: 0.724713
Cost after epoch 300: 0.495561
Cost after epoch 400: 0.336762
Cost after epoch 500: 0.228522
Cost after epoch 600: 0.173474
Cost after epoch 700: 0.117510
Cost after epoch 800: 0.089513
Cost after epoch 900: 0.062096
Train Accuracy: 0.9824074
Test Accuracy: 0.81666666
